In [1]:
# Multi-Robot APF Results Analysis
# This notebook analyzes the performance results from different test configurations

In [2]:
import json
import pandas as pd
import numpy as np
import os
from pathlib import Path
import matplotlib.pyplot as plt
# import seaborn as sns
from typing import Dict, List, Any

# Set up plotting style
plt.style.use('default')
# sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [3]:
def load_all_results(base_dir: str = '.') -> pd.DataFrame:
    results = []
    base_path = Path(base_dir)

    # Find all res.json files in subdirectories
    for json_file in base_path.rglob('res.json'):
        try:
            with open(json_file, 'r') as f:
                data = json.load(f)

            # Add folder name for reference
            data['folder_name'] = json_file.parent.name
            results.append(data)
            # print(f"Loaded: {json_file}")

        except Exception as e:
            print(f"Error loading {json_file}: {e}")

    if not results:
        print("No res.json files found!")
        return pd.DataFrame()

    df = pd.DataFrame(results)

    # #
    # print(f"\nLoaded {len(df)} result files")
    # print(f"Columns: {list(df.columns)}")

    return df

In [4]:
def refine_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    # Drop rows with nD == 2D
    df = df[df['nD'] != '2D']

    # Only keep relevant columns
    relevant_columns = ['map_id', 'n_robots', 'method', 'operation_time', 'total_length']
    df = df[relevant_columns]

    return df

In [5]:
# group by map_id and create tables for each map
def map_tables(df: pd.DataFrame) -> Dict[str, pd.DataFrame]:
    tables = {}
    for map_id, map_data in df.groupby('map_id'):
        tables[map_id] = map_data
    return tables

In [6]:
def create_performance_tables(tables: Dict[str, pd.DataFrame], metric: str) -> Dict[int, pd.DataFrame]:
    #  create table from df (which is for a specific map), columns are n_robots, index are methods, values are metric
    p_tables = {}
    for i, df in tables.items():
        table = df.pivot_table(index='method', columns='n_robots', values=metric, aggfunc=np.mean)
        p_tables[i] = table
    return p_tables

In [7]:
# Load all results
df_all = load_all_results('./tests')
df_all = refine_dataframe(df_all)
df_all.head(5)

,map_id,n_robots,method,operation_time,total_length
4,1,4,1,80.75,21.67
5,1,4,2,47.25,20.68
6,1,3,2,44.14,16.51
7,1,3,1,75.83,17.17


In [8]:
#
tables = map_tables(df_all)
tables = create_performance_tables(tables, metric='operation_time')
for map_id, table in tables.items():
    print(f"\nPerformance Table for Map ID: {map_id}")
    display(table)


Performance Table for Map ID: 1


n_robots,3,4
method,,
1,75.83,80.75
2,44.14,47.25


In [9]:
# def analyze_method_performance(df: pd.DataFrame):
#     """
#     Detailed analysis of method performance
#     """
#     print("=== METHOD PERFORMANCE ANALYSIS ===\n")

#     for map_id in sorted(df['map_id'].unique()):
#         map_data = df[df['map_id'] == map_id]
#         print(f"MAP {map_id} Analysis:")
#         print("-" * 30)

#         # Best method for each metric and robot count
#         metrics = ['operation_time', 'total_length']

#         for n_robots in sorted(map_data['n_robots'].unique()):
#             robot_data = map_data[map_data['n_robots'] == n_robots]
#             print(f"\n  {n_robots} Robots:")

#             for metric in metrics:
#                 if metric in robot_data.columns:
#                     best_method = robot_data.loc[robot_data[metric].idxmin(), 'method']
#                     best_value = robot_data[metric].min()
#                     worst_value = robot_data[metric].max()
#                     improvement = ((worst_value - best_value) / worst_value * 100)

#                     print(f"    {metric}: Method {best_method} (Best: {best_value:.2f}, "
#                           f"Improvement: {improvement:.1f}%)")

#         print("\n" + "="*50 + "\n")


# # Analyze method performance
# analyze_method_performance(df_all)